# MH of older workers, retirement age and working conditions

## Preprocessing EWCS data

In [1]:
import os
import sys

src_path = os.path.abspath("../")
sys.path.append(src_path)

from utils.common import *
from utils.ewcs import *
from utils.retirement import *
from utils.share import *

import_libraries()

## EWCS data

In [2]:
ewcs, meta_ewcs = pyreadstat.read_dta(
    "/Users/alexandralugova/Documents/GitHub/MH-old-workers/data/datasets/EWCS 1991-2015 UKDA ISCO.DTA"
)

In [3]:
ewcs = ewcs_preprocessing(ewcs, meta_ewcs)

In [4]:
df10, meta_df10 = pyreadstat.read_dta(
    "/Users/alexandralugova/Documents/GitHub/MH-old-workers/data/datasets/UKDA-2010/stata/stata11/ewcs_2010_version_ukda_6_dec_2011.dta"
)

In [5]:
soc = social_environment_index(ewcs, df10)
ewcs = ewcs.merge(soc, on=["id", "year"], how="left")

JQI social environment
        count       mean        std        min        25%        50%  \
year                                                                   
2010  30836.0  82.983566  14.530474  12.500000  75.000000  87.500000   
2015  13279.0  79.869788  15.768143   9.090909  72.727273  81.818182   

            75%    max  
year                    
2010  93.750000  100.0  
2015  90.909091  100.0  


In [6]:
pro = prospects_index(ewcs, df10)
ewcs = ewcs.merge(pro, on=["id", "year"], how="left")

JQI prospects
        count       mean        std  min   25%   50%   75%    max
year                                                             
2010  38556.0  55.655086  23.024819  0.0  37.5  50.0  75.0  100.0
2015  30073.0  57.977255  24.018895  0.0  50.0  62.5  75.0  100.0


In [7]:
int = intensity_index(ewcs, df10)
ewcs = ewcs.merge(int, on=["id", "year"], how="left")

JQI intensity
        count       mean        std       min        25%        50%  \
year                                                                  
2010  27063.0  56.353951  15.846348  4.651163  46.511628  58.139535   
2015  12947.0  46.274705  13.852596  2.325581  37.209302  46.511628   

            75%        max  
year                        
2010  67.441860  95.348837  
2015  55.813953  83.720930  


In [8]:
indexes = [col for col in ewcs.columns if col.startswith("jqi")]

for index in indexes:
    ewcs[f"{index}_weighted"] = ewcs[index] * ewcs["w4"]

In [9]:
ewcs = sum_wq_index(ewcs)

JQI working quality index
       count        mean        std         min         25%         50%  \
year                                                                      
2010  8692.0  422.290177  58.719339  181.754106  386.940634  428.266737   
2015  5234.0  415.680452  61.528587  133.185597  379.178912  422.432728   

             75%         max  
year                          
2010  464.941052  575.941286  
2015  460.123278  555.676416  


In [10]:
ewcs = ewcs[["country", "year", "isco"] + indexes]

ewcs = ewcs.groupby(["country", "year", "isco"]).mean().reset_index(drop=False)

ewcs = ewcs.dropna().reset_index(drop=True)

In [13]:
ewcs = interpolate2013(ewcs)

In [15]:
ewcs["year"] = ewcs["year"].replace({2010: 2011})

In [16]:
ewcs.isna().sum()

country                     0
year                        0
isco                        0
jqi_monthly_earnings        0
jqi_skills_discretion       0
jqi_physical_environment    0
jqi_working_time_quality    0
jqi_social_environment      0
jqi_prospects               0
jqi_intensity               0
dtype: int64

In [17]:
ewcs.to_csv(
    "/Users/alexandralugova/Documents/GitHub/MH-old-workers/data/datasets/results/work_quality_indexes_year_country.csv",
    index=False,
)